In [ ]:
import numpy as np
import networkx as nx
import time
import torch
import torch.nn.functional as F




        



# load dataset
testset = np.load('./testset_demo.npz')

test_features = torch.from_numpy(testset['F']/100)## normalized dBZ：(lgz)/10
test_labels = torch.from_numpy( (np.log10(testset['L']+1)) ) ## transformed precipitation labels：y=lg(y+1)
test_mask = torch.from_numpy(testset['M'])
del testset

num_feats = test_features.shape[2]
N_test  = test_features.shape[0]
N_val = np.floor(N_test/2)
N_val = N_val.astype(np.int32)

trainset = np.load('./trainset_demo.npz') 
train_features = torch.from_numpy(trainset['F']/100)
train_labels = torch.from_numpy( (np.log10(trainset['L']+1)) ) 
train_mask = torch.from_numpy(trainset['M'])

N_train  = train_features.shape[0]




In [ ]:
import dgl
from dgl import DGLGraph
from dgl.data import register_data_args, load_data
#from gat import GAT

from snat import SNAT
from output_layer import  output
from utils import EarlyStopping


In [ ]:

def mean_square_error(y,labels):
    mean_e= torch.sum((y-labels)**2)
    n = y.size(0)
    return mean_e, n

def b_ms_error(y,labels):
    s_e= torch.sum(labels*(y-labels)**2)
    n = labels.sum()
    return s_e, n


def evaluate0(model, features, labels, mask, criterion):
    
    model.eval()
    with torch.no_grad():
        y = model(features)
        y = y[mask]
        y = y.flatten(0) 
        y = 10**y-1
        labels = 10**labels-1
        return criterion(y, labels[mask])


gpu= 0 #gpu device
#hyperparameters

threhold = 0.4## boundary for dividing node types
num_heads=1
num_out_heads=1
num_layers = 6
num_hidden= 16
out_dim= 64
early_stop= True
fastmode= False

if gpu < 0:
    cuda = False
else:
    cuda = True
    torch.cuda.set_device(gpu)

from dgl.data.utils import load_graphs
g = load_graphs("./Graph.bin",[0])
g = g[0][0]

# create model
heads = torch.zeros(num_layers+1,dtype=torch.int32)
nhidT = torch.zeros(num_layers+1,dtype=torch.int32)
heads[0:num_layers]=num_heads
heads[-1]=num_out_heads
nhidT[0]= 16
nhidT[1]=16
nhidT[2]=16
nhidT[3]=16
nhidT[4]=16
nhidT[5]=16
nhidT[6]=16
model = SNAT(g,
             threhold,
            num_layers,
            num_feats,
            num_hidden,
            out_dim,
            heads=heads,
            num_hiddenT=nhidT,
            feat_drop =0,
            attn_drop = 0,
            negative_slope=0.2,
            activation= torch.tanh,  ##activation function for intermidiate layers
            residual=False)


if cuda:
    model.cuda()
    
print(model)
if early_stop:
    stopper = EarlyStopping(patience=30)

loss_fcn = torch.nn.MSELoss(reduction='none')   

# use optimizer
optimizer1 = torch.optim.Adam(
    model.parameters(), lr=1e-4)##不知道对不对
#optimizer1 = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

dur = []
for epoch in range(200):

    val_error=0
    loss_m = 0
    N_total = 0
    if epoch >= 3:
        t0 = time.time()
    rand_n=torch.randperm(N_train) 
    batchsize= 1  
    nbatches = int(N_train/batchsize/3) #use 1/3 of the training data every epoch
    N_nodes_train= 0
    for batch in range(nbatches):
        N_ba = 0
        for n in range(batchsize): 
            model.train()

            # forward
            index_n = rand_n[n+batchsize*batch]
            H0 = train_features[index_n,:,:]
            index_0rd = (H0==0).flatten()
            train_labels[index_n,index_0rd]=0##去除异常值
            index_2rd = (train_labels[index_n]>2).flatten()
            train_mask[index_n,index_2rd]=False ##去除太大的zhi
            labels = train_labels[index_n,train_mask[index_n,:]]
            if cuda== True:
                H0=H0.cuda()
                labels=labels.cuda()
            N_t = labels.size(0)
            y = model(H0) 
            index_0 = (labels) ==0 ###标签为0的索引
            W_nodes = 10**(labels)  ##  weigtht for none zero label: W
            W_nodes[index_0]=0.1  ## weight for zero label: W0
            loss = loss_fcn( y[train_mask[index_n,:],:].flatten(),
                            labels )  ###compute the loss for every (labeled) node

            loss = loss*W_nodes 
            loss_w = loss.mean() ## average loss among all nodes
            loss_m += loss.sum().item()
            loss_w.backward()
            N_ba +=N_t
        optimizer1.step() 
        optimizer1.zero_grad()
        N_nodes_train += N_ba           
    for n in range(N_val):
        model.eval()
        index_0rd = (test_features[n]==0).flatten()
        test_labels[n,index_0rd]=0##去除异常值
        mask = (10**test_labels[n]-1)>0
        index_2rd = (test_labels[n]>2).flatten()
        test_mask[n,index_2rd]=False ##去除太大的zhi
        mask = mask&test_mask[n]
        test_F=test_features[n]
        test_L=test_labels[n]
        if cuda==True:
            test_F=test_F.cuda()
            test_L=test_L.cuda()
        val_error0, N0 = evaluate0(model, test_F, test_L, mask, b_ms_error)#
        val_error += val_error0
        N_total += N0
        
    val_error/=N_total
    if early_stop: 
        if stopper.step(-val_error, model, 'CNGAT_16_6_64.pt'):   
            break
    loss_m = loss_m/N_nodes_train
    if epoch >= 3:
        dur.append(time.time() - t0)
    print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.6f} |val_error{:.4f}".
              format(epoch, np.mean(dur), loss_m, val_error))
